In [1]:
import numpy as np

In [2]:
a = np.identity(3)

In [5]:
np.add(a,3)

array([[ 4.,  3.,  3.],
       [ 3.,  4.,  3.],
       [ 3.,  3.,  4.]])

In [11]:
b = np.vstack((np.array([1,2]).reshape(-1,1),1))